# A Comparison of State Use of SFLRF Funds for Vaccination Programs and Vaccination Rates in Each State



### Data Sources:
CDC - "COVID-19 Vaccinations in the United States, Jurisdiction"
csv downloaded 5/11/23
https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-Jurisdi/unsk-b7fc

EARN/EPI - "EARN SLFRF Workbook for Q4 2022" compiled by Dave Kamper of the Economic Policy Institute (dkamper@epi.org) from Treasury reports by states and local jurisidictions who received funding, and other data sources as detailed in the workbook.

In [ ]:
# import dependencies and setup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pprint import pprint
from pathlib import Path


## Production Code (Team: Put your code here after it is complete and ready to go)

## Evan Work Area

In [ ]:
# import dependencies and setup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pprint import pprint
from pathlib import Path

In [4]:
# Load csv file(s)
all_states_sheet = Path("Resources/EARN_all_states.csv")


# Read csv file(s) as a DataFrame
all_states_df = pd.read_csv(all_states_sheet)


# merge csv files (if applicable)


# preview the raw DataFrame
all_states_df.head()

In [ ]:
# Brainstorm a list of words to filter the 'Project Description' column by.
    ## this list will be used to filter that column so that we are only working with projects that
    ## are actually covid related.
    
search_term_list = #[covid, vaccine, ...]


In [ ]:
# Filter the dataframe column 'Project Description'

vaccine_projects_df = #all_states_df.<filter function syntax here> ...if ['Project Description'] == search_term_list

# preview the 'filtered' dataframe:
vaccine_projects_df.head()


In [ ]:
# Try to group the filtered dataframe by state, summing applicable $ value columns
    ## if we get errors, then we need to clean columns causing errors. 
    ## eg) 'Adopted Budget' column has values containing "-". This might prevent the .sum() function from working

state_spending_df = # vaccine_projects_df.groupby(['State/Territory']).sum(['Adopted Budget', 'Total Cumulative Obligations', 'Total Cumulative Expenditures'])



## Data Exploration and Cleanup:
- Describe here the group's data sets and how they were cleaned for analysis

# Greg Work Area

### CDC Data

In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import scipy.stats as st
import seaborn as sns

In [ ]:
#Import vaccination data from csv
vac_df = pd.read_csv('COVID-19_Vaccinations_in_the_United_States_Jurisdiction.csv')

In [ ]:
#function formats the CDC dataframe for US jursdictions - see below for input formats
def CDC_format(df,key,date,add_str,drop_values): #key, filt -> str; length -> int; columns, drop_values -> list
    df = df.dropna(subset=[key])
    df = df[vac_df['Date'] == date]
    df = df[~df[key].isin(drop_values)]
    df[key] = add_str + vac_df[key].astype(str)
    df.reset_index(drop = True, inplace = True)
    return df

In [ ]:
drop = ['DD2','FM','AS','VI','BP2','IH2','GU','PN','PR','VA2','PW','US','MP','MH']
vac_df = CDC_format(vac_df,'Location','12/28/2022','US_',drop)

In [ ]:
vac_df

In [ ]:
### Google vac data

In [ ]:
#Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import scipy.stats as st
import seaborn as sns
import cartopy.crs as ccrs
import geoviews as gv # noqa
import pyproj
import geopandas as gpd
import hvplot.pandas
import plotly.express as px

In [ ]:
#Import vaccination data from google api
vac_df = pd.read_csv('https://storage.googleapis.com/covid19-open-data/v3/latest/vaccinations.csv')

In [ ]:
#function formats the google dataframes - see below for input formats
def google_format(df,key,filt,length,columns,drop_values): #key, filt -> str; length -> int; columns, drop_values -> list
    df = df.dropna(subset=[key])
    df = df[df[key].str.contains(filt)]
    mask = (df[key].str.len() == length)
    df = df.loc[mask]
    df = df[columns]
    df = df[~df[key].isin(drop_values)]
    df.reset_index(drop = True, inplace = True)
    return df

In [ ]:
#Input values for vaccination data
drop = ['US_AS','US_GU','US_MP','US_PR','US_VI']
cols = ['date','location_key','cumulative_persons_fully_vaccinated','new_persons_vaccinated','new_persons_fully_vaccinated']
loc_key = 'location_key'
contains = 'US_'

In [ ]:
#formatting vaccination data
vac_df = google_format(vac_df, loc_key, contains, 5, cols, drop)

In [ ]:
mylist = ['Orange','Apple'] #Keywords search
pattern = '|'.join(mylist)
vac_df.location_key.str.contains(pattern)

In [ ]:
#reading demographic data
dem_df = pd.read_csv('demographics.csv')

In [ ]:
dem_df

In [ ]:
dcols = ['location_key','population']

In [ ]:
#formatting demographic data
dem_df = google_format(dem_df, loc_key, contains, 5, dcols, drop)

In [ ]:
#reading epidemeology data
epi_df = pd.read_csv('https://storage.googleapis.com/covid19-open-data/v3/latest/epidemiology.csv')

In [ ]:
ecols = ['location_key','cumulative_confirmed','cumulative_deceased','cumulative_recovered']

In [ ]:
#formatting epidemeology data
epi_df = google_format(epi_df, loc_key, contains, 5, ecols, drop)

In [ ]:
loc_key = pd.read_csv('https://storage.googleapis.com/covid19-open-data/v3/location/US.csv')

In [ ]:
AK_vac_df = US_vac_df[US_vac_df['location_key'].str.contains('US_AK')]

In [ ]:
#Looking at only one state - this can be skipped
AK_total = AK_vac_df['cumulative_persons_fully_vaccinated'].iloc[1:len(AK_vac_df)].sum()
AK_total

In [ ]:
#we don't need this at the moment, can be skipped
def swap_rows(df, i1, i2): #Keep this!!!
    a, b = df.iloc[i1, :].copy(), df.iloc[i2, :].copy()
    df.iloc[i1, :], df.iloc[i2, :] = b, a
    return df

In [ ]:
#merging dataframes
total_df = vac_df.merge(dem_df, how = 'inner',on = 'location_key')

In [ ]:

total_df['percent_fully_vaccinated'] = (total_df['cumulative_persons_fully_vaccinated']/total_df['population'])*100
total_df.sort_values('percent_fully_vaccinated', ascending = False)

In [ ]:
#merging dataframes
total_df = total_df.merge(epi_df, how = 'inner',on = 'location_key')

In [ ]:
total_df['percent_death_rate_by_case'] = (total_df['cumulative_deceased']/total_df['cumulative_confirmed'])*100

In [ ]:
total_df['percent_death_rate_per_capita'] = (total_df['cumulative_deceased']/total_df['population'])*100

In [ ]:
total_df['percent_confirmed'] = (total_df['cumulative_confirmed']/total_df['population'])*100

In [ ]:
total_df['state_code'] = total_df.location_key.str.replace('US_','') #adding the state code for the plotly function

In [ ]:
total_df.sort_values('percent_fully_vaccinated', ascending = False)

In [ ]:
#function for regression plots
def reg(df,x,y,x_text,y_text):    
    lm = st.linregress(x = df[x], y = df[y])
    data_fit = lm[0]*df[x] + lm[1]
    fit_df = pd.DataFrame({'x': df[x], 'fitted': data_fit})
    ax = sns.scatterplot(data = df, x = x, y = y)
    #ax = df.plot.scatter(y = y, x = x, s = 30)
    print(f"The r-value is: {lm[2]}")
    fit_df.plot.line(x = 'x', y = 'fitted', color = 'red', ax=ax, legend = None, xlabel = x)
    plt.text(x_text,y_text,f"y = {'%.2f' %lm[0]}x + {'%.1f' %lm[1]}", color = 'red', fontsize = 16)

In [ ]:
reg(total_df,'percent_fully_vaccinated','percent_death_rate_by_case',50,0.6)

In [ ]:
reg(total_df,'percent_fully_vaccinated','percent_death_rate_per_capita',50,0.15)

In [ ]:
reg(total_df,'percent_fully_vaccinated','percent_confirmed',50,20)

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

world.hvplot(c='country', geo=True)

In [ ]:
#generating map of us states - you need to specify the color variable as one of the dataframe columns 
fig = px.choropleth(total_df,
                    locations='state_code', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='percent_death_rate_per_capita',
                    color_continuous_scale="blues" 
                    )
# fig.add_scattergeo(
#     locations=total_df['state_code'],
#     locationmode="USA-states", 
#     text=total_df['state_code'],
#     mode='text',
# )
fig.show()

# Joanna Work Area

# Kendal Work Area

In [ ]:
#putting Greg's code down here so I can run my area independently of the rest of the sheet without error
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy.stats import linregress
import scipy.stats as st
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
#Import vaccination data from csv
vac_df = pd.read_csv('COVID-19_Vaccinations_in_the_United_States_Jurisdiction.csv')

In [ ]:
#function formats the CDC dataframe for US jursdictions - see below for input formats
def CDC_format(df,key,date,add_str,drop_values): #key, filt -> str; length -> int; columns, drop_values -> list
    df = df.dropna(subset=[key])
    df = df[vac_df['Date'] == date]
    df = df[~df[key].isin(drop_values)]
    df[key] = add_str + vac_df[key].astype(str)
    df.reset_index(drop = True, inplace = True)
    return df

In [ ]:
#drop non-state territories from dataframe
drop = ['DD2','FM','AS','VI','BP2','IH2','GU','PN','PR','VA2','PW','US','MP','MH']
vac_df = CDC_format(vac_df,'Location','12/28/2022','US_',drop)

In [ ]:
#change location to match state code for choropleth maps
vac_df['Location'] = vac_df['Location'].str.replace('US_', '')
vac_df = vac_df.rename(columns={'Series_Complete_Pop_Pct':'Percentage of Population Fully Vaccinated',
                            'Administered_Dose1_Pop_Pct':'% of Population with at least 1 dose',
                            'Series_Complete_65PlusPop_Pct': '% of Population Fully Vaccinated (65+)',
                            'Bivalent_Booster_65Plus_Pop_Pct': '% of Population with bivalent booster (65+)'}) 

In [ ]:
#Go.choropleth method (https://plotly.com/python/choropleth-maps/)
fig = go.Figure(data=go.Choropleth(
    locations=vac_df['Location'], # Spatial coordinates
    z = vac_df['Percentage of Population Fully Vaccinated'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Viridis',
    colorbar_title = "Percentage of Population Fully Vaccinated",
))

fig.update_layout(
    title_text = 'Vaccination Rates by State',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

In [ ]:
fig_complete_total_pop = px.choropleth(vac_df,
                    locations='Location',
                    locationmode="USA-states",
                    scope="usa",
                    color='Percentage of Population Fully Vaccinated',
                    color_continuous_scale="aggrnyl",
                    )

In [ ]:
fig_at_least_1 = px.choropleth(vac_df,
                    locations='Location',
                    locationmode="USA-states",
                    scope="usa",
                    color='% of Population with at least 1 dose',
                    color_continuous_scale="twilight",
                    )

In [ ]:
fig_complete_65_plus = px.choropleth(vac_df,
                    locations='Location',
                    locationmode="USA-states",
                    scope="usa",
                    color='% of Population Fully Vaccinated (65+)',
                    color_continuous_scale="algae",                 
                    )

In [ ]:
fig_bivalent_booster_65 = px.choropleth(vac_df,
                    locations='Location',
                    locationmode="USA-states",
                    scope="usa",
                    color='% of Population with bivalent booster (65+)',
                    color_continuous_scale="icefire",                  
                    )

In [ ]:
fig_complete_total_pop

In [ ]:
fig_at_least_1

In [ ]:
fig_complete_65_plus

In [ ]:
fig_bivalent_booster_65

In [ ]:
#Making percent fully vaccinated into a list
percent_fully_vaccinated = vac_df['Percentage of Population Fully Vaccinated'].to_numpy()
print(percent_fully_vaccinated)

In [ ]:
#boxplots showing spread of data across all 50 states and DC for selected columns
boxplot = vac_df.boxplot(column=['Percentage of Population Fully Vaccinated', 
                                 '% of Population with at least 1 dose', 
                                 '% of Population Fully Vaccinated (65+)', 
                                 '% of Population with bivalent booster (65+)'], 
                         rot=45,
                         grid=True,
                         figsize = (15,10),
                        )
plt.title("Distribution of Vaccination Rates Across U.S. States")

In [ ]:
#summary statistics for selected columns (across all 50 states and DC) 
vac_df_short = vac_df[['Series_Complete_Pop_Pct', 'Administered_Dose1_Pop_Pct', 'Series_Complete_65PlusPop_Pct', 'Bivalent_Booster_65Plus_Pop_Pct']]

vac_df_short.describe()

# Sarah Work Area

# Aaliyah Work Area